## Import libraries

In [157]:
import pandas as pd
import sys
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager
import matplotlib.font_manager as fm
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportion_confint
from tableone import TableOne, load_dataset


## Display Python and Pandas version numbers

In [ ]:

print('Python version:', sys.version)
print('pandas version:', pd.__version__)

## Import data and display its structure

In [ ]:
# Read the pickled data

df = pd.read_pickle("../data/muscle_data/clean_data.pkl")

pd.set_option("display.max_columns", None)

# Check the data
print(df.head())
print(df.shape)
df.columns

## Create Table 1

In [ ]:
df.columns

In [161]:
columns = ['age', 'sex', 'bmi', 'ppi_status', 'grip_strength_mean',
           'walking_speed_4m', 'armchair_push_pull_success']

categorical = ['sex', 'ppi_status', 'armchair_push_pull_success']

continuous = ['age', 'bmi', 'grip_strength_mean', 'walking_speed_4m']

groupby = 'ppi_status'

rename={'age': 'Age'}



In [ ]:
mytable = TableOne(df, columns=columns,
                  categorical=categorical,
                  continuous=continuous,
                  groupby=groupby,
                  rename=rename,
                  pval_adjust=None,
                  pval=True)

mytable


In [ ]:
print(mytable.tabulate(tablefmt = "none"))

mytable.to_excel('tables/table1.xlsx')

## Prepare separate datasets according to PPI status for testing difference

In [164]:
# Prepare data for testing with mean grip strength
ppi_grip_mean = df[df['ppi_status'] == 'PPI User']['grip_strength_mean']
no_ppi_grip_mean = df[df['ppi_status'] == 'No PPI']['grip_strength_mean']
 

# Create subsets for walking speed analysis
walking_speed_ppi = df[df['ppi_status'] == 'PPI User']['walking_speed_4m']
walking_speed_no_ppi = df[df['ppi_status'] == 'No PPI']['walking_speed_4m']


## Run t-test for mean grip strength

In [ ]:
# Statistical tests with mean grip strength
# t-test
t_stat, p_value = stats.ttest_ind(ppi_grip_mean, no_ppi_grip_mean)
print("Independent t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

mean_grip_t_test_p_value = p_value


## Run t-test for walking speed

In [ ]:
# Statistical tests with mean grip strength
# t-test
t_stat, p_value = stats.ttest_ind(walking_speed_ppi, walking_speed_no_ppi, equal_var=False)

print("Independent t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

walking_speed_t_test_p_value = p_value

## Test whether the desired font exists

In [ ]:
# First, let's check if Rosario is already available
available_fonts = [f.name for f in matplotlib.font_manager.fontManager.ttflist]
print("Fonts with 'Rosario' in name:")
print([f for f in available_fonts if 'Rosario' in f])

## Create plots with the Plotnine package

In [ ]:
figure1 = (ggplot(df, aes(x='ppi_status', y='grip_strength_mean', fill='ppi_status')) +
 scale_fill_brewer(palette=1) +
 geom_boxplot() +
 theme_classic(base_size = 26) +  # Classic theme
 labs(title='Grip Strength',
      x='PPI Status',
      y='Grip Strength (kg)') +

 # Add horizontal line
 annotate('segment', x=1.3, xend=1.7, y=42, yend=42, color='black') +

 # Add left vertical whisker
 annotate('segment', x=1.3, xend=1.3, y=40, yend=42, color='black') +
 # Add right vertical whisker
 annotate('segment', x=1.7, xend=1.7, y=40, yend=42, color='black') +

 annotate('text', 
          x=1.5,
          y=45, 
          label=f'P = {mean_grip_t_test_p_value:.2f}',
          size=18,
          family="Rosario") +

 theme(figure_size=(10, 6),
       legend_position='none',
       legend_title=element_blank(),
       text=element_text(family='Rosario'),
       axis_text=element_text(family='Rosario'),
       title=element_text(margin={"t": 30, "b": 30}, family='Rosario'),
       axis_title_y=element_text(margin={"r": 10}),
       axis_title_x=element_text(margin={"t": 10}))  # Increase right margin of y-axis title

)

# Export the plot
figure1.save("figures/Fig1.pdf")
figure1.save("figures/Fig1.tiff", dpi=600, pil_kwargs={"compression": "tiff_lzw"})


In [ ]:
# Display the plot
figure1

In [ ]:
figure2 = (ggplot(df, aes(x='ppi_status', y='walking_speed_4m', fill='ppi_status')) +
 scale_fill_brewer(palette=1) +
 geom_boxplot() +
 theme_classic(base_size = 26) +  # Classic theme
 labs(title='4-Meter Walking Time',
      x='PPI Status',
      y='4-Meter Walking Time (s)') +

 # Add horizontal line
 annotate('segment', x=1.3, xend=1.7, y=5.5, yend=5.5, color='black') +

 # Add left vertical whisker
 annotate('segment', x=1.3, xend=1.3, y=5.5, yend=5, color='black') +
 # Add right vertical whisker
 annotate('segment', x=1.7, xend=1.7, y=5.5, yend=5, color='black') +

 annotate('text', 
          x=1.5,
          y=6.5, 
          label=f'P = {walking_speed_t_test_p_value:.2f}',
          size=18,
          family="Rosario") +

 theme(figure_size=(10, 6),
       legend_position='none',
       legend_title=element_blank(),
       text=element_text(family='Rosario'),
       axis_text=element_text(family='Rosario'),
       title=element_text(margin={"t": 30, "b": 30}, family='Rosario'),
       axis_title_y=element_text(margin={"r": 10}),
       axis_title_x=element_text(margin={"t": 10}))  # Increase right margin of y-axis title
)

# Export the plot
figure2.save("figures/Fig2.pdf")
figure2.save("figures/Fig2.tiff", dpi=600, pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# Display the plot
figure2

In [172]:
# Calculate percentages and CIs for each group
def calc_stats(df, group_col, success_col):
    stats_dict = {}
    for group in df[group_col].unique():
        group_data = df[df[group_col] == group]
        success_count = (group_data[success_col] == "Success").sum()
        total_count = len(group_data)
        pct = (success_count / total_count) * 100
        
        # Calculate Wilson confidence intervals
        ci_lower, ci_upper = proportion_confint(success_count, total_count, alpha=0.05, method='wilson')
        ci_lower *= 100  # Convert to percentage
        ci_upper *= 100  # Convert to percentage
        
        stats_dict[group] = {'pct': pct, 'ci_lower': ci_lower, 'ci_upper': ci_upper}
    
    # Create DataFrame for plotting
    plot_data = pd.DataFrame({
        'ppi_status': list(stats_dict.keys()),
        'pct': [d['pct'] for d in stats_dict.values()],
        'ci_lower': [d['ci_lower'] for d in stats_dict.values()],
        'ci_upper': [d['ci_upper'] for d in stats_dict.values()]
    })
    
    return plot_data

# Calculate chi-square test
def calc_chi_square(df, group_col, success_col):
    contingency = pd.crosstab(df[group_col], df[success_col])
    chi2, p_value, _, _ = chi2_contingency(contingency)
    return p_value

# Prepare data for plotting
plot_data = calc_stats(df, 'ppi_status', 'armchair_push_pull_success')
p_value = calc_chi_square(df, 'ppi_status', 'armchair_push_pull_success')

armchair_push_pull_success_p_value = p_value

In [ ]:
figure3 = (ggplot(plot_data, aes(x='ppi_status', y='pct', fill='ppi_status')) +
 geom_bar(stat="identity", position="dodge", width=0.6, color="black") +
 scale_y_continuous(limits=(0, 100), expand=(0,0)) +
 labs(x="PPI Status", 
      y="Percentage (%)",
      title="Proportion of Successful Armchair Push-Pull Test") +
 theme_classic(base_size=16, base_family="Rosario") +  # Added base_family="Rosario"
 theme(
     plot_title=element_text(ha="center", weight="bold", margin={"t": 20, "b": 20}, family="Rosario"),
     axis_text=element_text(family="Rosario"),
     axis_title=element_text(weight="bold", family="Rosario"),
     axis_title_x=element_text(margin={"t": 20}, family="Rosario"),
     axis_title_y=element_text(margin={"r": 15}, family="Rosario"),
     axis_text_x=element_text(margin={"t": 10}, family="Rosario"),
     axis_text_y=element_text(margin={"r": 5}, family="Rosario"),
     legend_position="none"
 ) +
 geom_errorbar(aes(ymin='ci_lower', ymax='ci_upper'), 
               width=0.1, color="black", alpha=0.9, size=0.5) +

 # Add horizontal line
 annotate('segment', x=1, xend=2, y=99, yend=99, color='black') +

 # Add left vertical whisker
 annotate('segment', x=1.0015, xend=1.0015, y=98, yend=99, color='black') +
 # Add right vertical whisker
 annotate('segment', x=1.9983, xend=1.9983, y=98, yend=99, color='black') +

 annotate('text', 
          x=1.5,
          y=93, 
          label=f'P = {armchair_push_pull_success_p_value:.2f}',
          size=12,
          family="Rosario") +  # Added family="Rosario" here too
 #scale_fill_manual(values=['#D8C3BA', '#BB9B90'])
 scale_fill_brewer(palette=1)
)


# Export the plot
figure3.save("figures/Fig3.pdf")
figure3.save("figures/Fig3.tiff", dpi=600, pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# Display the plot
figure3


## Grip strength, simple linear regression model

In [ ]:

model = smf.ols("grip_strength_mean ~ C(ppi_status)", data=df).fit()

print(model.summary())


## Grip strength, multiple linear regression model

In [ ]:


model = smf.ols("grip_strength_mean ~ C(ppi_status) + C(sex) + age + bmi", data=df).fit()

print(model.summary())



## 4-minute walking test, simple linear regression model

In [ ]:

model = smf.ols("walking_speed_4m ~ C(ppi_status)", data=df).fit()

print(model.summary())


## 4-minute walking test, multiple linear regression model

In [ ]:
model = smf.ols("walking_speed_4m ~ C(ppi_status) + C(sex) + age + bmi", data=df).fit()

print(model.summary())


## Armchair push-pull test, simple regression model

In [ ]:
# Logistic regression model

# First, create binary variables (0/1) for our categorical variables
df['armchair_success_binary'] = (df['armchair_push_pull_success'] == 'Success').astype(int)

# Veriry the structure of the data if needed
# df[['armchair_success_binary', 'armchair_push_pull_success']].head()

# Fit the logistic regression model
model = smf.logit("armchair_success_binary ~ C(ppi_status)", data=df).fit()

# Print summary
print(model.summary())

# Calculate odds ratio and confidence intervals
odds_ratio = np.exp(model.params[1])
conf_int = np.exp(model.conf_int().iloc[1])

print("\nOdds Ratio Analysis:")
print(f"Odds Ratio: {odds_ratio:.2f}")
print(f"95% CI: ({conf_int[0]:.2f}, {conf_int[1]:.2f})")



## Armchair push-pull test, multiple regression model

In [ ]:
# Logistic regression model

# First, create binary variables (0/1) for our categorical variables
df['armchair_success_binary'] = (df['armchair_push_pull_success'] == 'Success').astype(int)

# Veriry the structure of the data if needed
# df[['armchair_success_binary', 'armchair_push_pull_success', 'ppi_status', 'ppi_user_binary']].head()

# Fit the logistic regression model
model = smf.logit("armchair_success_binary ~ C(ppi_status) + C(sex) + age + bmi", data=df).fit()

# Print summary
print(model.summary())

# Calculate odds ratio and confidence intervals
odds_ratio = np.exp(model.params[1])
conf_int = np.exp(model.conf_int().iloc[1])

print("\nOdds Ratio Analysis:")
print(f"Odds Ratio: {odds_ratio:.2f}")
print(f"95% CI: ({conf_int[0]:.2f}, {conf_int[1]:.2f})")

